Offset sight ballistics


In [11]:
import numpy as np
import pandas as pd

# Load the autoreload extension
%load_ext autoreload

# Set autoreload mode
%autoreload 2
#%reload_ext autoreload
from util import read_parameters_from_file
import ballistics as b
import util as util


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
shooting_angle = 0 # degrees

# Offset sight
offset_hob=55 / 1000 # m
offset_d_zero=36 # m
offset_mrad_per_click = 0.13

In [13]:
# Define bullet parameters
try:
    params = read_parameters_from_file()
    barrel_angle = params.barrel_angle
    v0 = params.initial_velocity
    drag_coefficient_g1 = params.drag_coefficient
    h = params.height
    bullet_mass = params.bullet_mass
    bullet_area = params.bullet_cross_sectional_area
    temp = params.temperature
    pressure = params.pressure
    humidity = params.humidity
except AttributeError as e:
    print(f"An error occurred while reading parameters: {e}")
    # Handle the error (e.g., use default values or terminate the program)
    exit(1)

air_density = b.calculate_air_density(temp, pressure, humidity)
util.set_air_density(air_density)
bullet_weight = bullet_mass

Loaded parameters


In [14]:
offset_distances = np.arange(0, 100 + 12, 12)
offset_tbr_distances = b.calculate_true_ballistic_ranges(offset_distances, shooting_angle) # adjusted for shooting angle

offset_barrel_angle = b.calibrate_zero(v0, offset_d_zero, drag_coefficient_g1, bullet_weight, bullet_area, offset_hob)

offset_velocities = b.calculate_velocities(v0, drag_coefficient_g1, bullet_weight, bullet_area, offset_tbr_distances, offset_barrel_angle)

offset_pois = np.array(b.calculate_pois(v0, drag_coefficient_g1, bullet_weight, bullet_area, offset_hob, offset_barrel_angle, offset_tbr_distances))

offset_pois_mrad = np.array(b.calculate_mrads(offset_tbr_distances, offset_pois))

offset_clicks = np.round(np.array(list(map(lambda x: x / offset_mrad_per_click, offset_pois_mrad[offset_distances <= 100])))).astype(int)

do = {'drop (m)': offset_pois[offset_distances <= 100].round(2), 'mrads': offset_pois_mrad[offset_distances <= 100].round(2),
      'clicks (h)': offset_clicks}
dfo = pd.DataFrame(data=do, index=offset_distances[offset_distances <= 100])
styled_dfo = dfo.style.set_caption("Offset sight ballistics")
display(styled_dfo)


Calibrating zero...
Calibration complete. Barrel angle: 0.000210149935711733


,drop (m),mrads,clicks (h)
0,nan,0.000000,0
12,0.000000,-0.140000,-1
24,0.000000,-0.070000,-1
36,0.000000,-0.000000,0
48,-0.000000,0.070000,1
60,-0.010000,0.140000,1
72,-0.010000,0.200000,2
84,-0.020000,0.270000,2
96,-0.030000,0.340000,3
